# RHESSI flare from temporal decomposition paper
Here we analyze the 2011 Jul 30 flare observed by RHESSI from the temporal decomposition paper.

We don't perform spectroscopy; we just demonstrate the decomposition technique.

## How to: set up this package
Please see the `readme.md` file in the root of this repository.

## How to: get data
The RHESSI flare is downloaded using SSWIDL routines.
An appropriate SSWIDL license and installation is required to parse the level zero RHESSI data into a spectrogram.
The following snippet downloads the spectrum file and saves it to a .fits:
```idl
; see https://sprg.ssl.berkeley.edu/~tohban/wiki/index.php/Creating_a_Spectrum_File_Using_the_HESSI_GUI
; guide on how to do this

; config me
t_int = ['30-Jul-2011T01:50:00.000', '30-Jul-2011 02:20:00.000']
; see here for binning information
; https://hesperia.gsfc.nasa.gov/ssw/hessi/dbase/spec_resp/energy_binning.txt
start_energy = alog10(4)
end_energy = alog10(200)
num = 20
expons = dindgen(num) / (num - 1) * (end_energy - start_energy) + start_energy
ebins = 10^expons
; add a bin at the end for the background
energy_bins = [ebins, [500, 800]]

; use at least 4s wide bins to prevent rotation effects
time_bin_width = 4


name = 'trevor-flare-30-jul-2011-logspace-bkg'
specfile = name + '_spec.fits'
srmfile = name + '_srm.fits'

search_network, /enable

spec_obj = hsi_spectrum()
spec_obj-> set, obs_time_interval=t_int
spec_obj-> set, decimation_correct=1    
spec_obj-> set, rear_decimation_correct= 0    
spec_obj-> set, pileup_correct=0    
spec_obj-> set, /sp_semi_calibrated

; "detectors 3, 4, and 9 have the best energy resolution of all the detectors" -- from the wiki page
; whatever
spec_obj-> set, seg_index_mask=[1B, 1B, 1B, 1B, 1B, 1B, 1B, 1B, 1B, $
			    0B, 0B, 0B, 0B, 0B, 0B, 0B, 0B, 0B] 
spec_obj-> set, sp_chan_binning=0
spec_obj-> set, sp_chan_max=0
spec_obj-> set, sp_chan_min=0
spec_obj-> set, sp_data_unit='Flux'
spec_obj-> set, sp_energy_binning=energy_bins
spec_obj-> set, sp_semi_calibrated=0B
spec_obj-> set, sp_time_interval=time_bin_width
spec_obj-> set, sum_flag=1
spec_obj-> set, use_flare_xyoffset=1

spec_obj->filewrite, /buildsrm, all_simplify=0, srmfile=srmfile, specfile=specfile
print, 'done'
```

In [ ]:
from sunkit_spex.extern import rhessi
from yaff.plotting import stairs_with_error

import astropy.units as u
import astropy.time as atime
from astropy.visualization import quantity_support
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tedec import decomp

## Load in the data and perform the temporal decomposition

### First, load the data and slice out the time range we want

In [ ]:
def nearest(a, v):
    return np.argmin(np.abs(a - v))


spectrum_file = "trevor-flare-30-jul-2011-logspace-bkg_spec.fits"
srm_file = "trevor-flare-30-jul-2011-logspace-bkg_srm.fits"

data = rhessi.RhessiLoader(spectrum_fn=spectrum_file, srm_fn=srm_file)

start_event_time = atime.Time("2011-07-30T02:08:20")
end_event_time = atime.Time("2011-07-30T02:10:20")
start_background_time = atime.Time("2011-07-30T01:54:00")
end_background_time = atime.Time("2011-07-30T01:56:00")

data.update_event_times(start_event_time, end_event_time)
data.update_background_times(start_background_time, end_background_time)

### Slice out the data from sunkit-spex format into a simplified data structure

In [ ]:
def prep_rhessi_data(rl: rhessi.RhessiLoader) -> dict[str, u.Quantity]:
    """Take a sunkit_spex object and extract the data we want from it"""
    tbins = atime.Time(
        np.concatenate(
            (rl._spectrum["time_bins"][:, 0], [rl._spectrum["time_bins"][-1, -1]])
        )
    )
    ebins = np.unique(rl["count_channel_bins"].flatten()) << u.keV

    cts = rl._spectrum["counts"] / rl._spectrum["livetime"]
    err = rl._spectrum["counts_err"] / rl._spectrum["livetime"]

    tai, tbi = (
        nearest(tbins, atime.Time(rl._start_event_time)),
        nearest(tbins, atime.Time(rl._end_event_time)),
    )
    cut_cts = cts[tai:tbi] << u.ct
    cut_err = err[tai:tbi] << u.ct
    cut_tbins = tbins[tai : tbi + 1]

    return {
        "time_bins": cut_tbins,
        "energy_bins": ebins,
        "cts": cut_cts.T,
        "cts_err": cut_err.T,
    }

In [ ]:
sliced = prep_rhessi_data(data)

### Then, perform the decomposition

In [ ]:
# We want to apply any sort of lightcurve summations by indexing the
# energy midpoints, as they have the same shape as the counts.
energy_mids = sliced["energy_bins"][:-1] + np.diff(sliced["energy_bins"]) / 2

# Template energies for (thermal, nonthermal, background).
TH_ENG = 5.5 << u.keV
NTH_ENG = 81 << u.keV
BKG_ENG = 320 << u.keV

# Find the indices of the count bins that are closest to our desired energies
th_idx = nearest(TH_ENG, energy_mids)
nth_idx = nearest(NTH_ENG, energy_mids)
bkg_idx = nearest(BKG_ENG, energy_mids)

# Package up all of our data into a DataPacket
pack = decomp.DataPacket(
    data=sliced["cts"],
    basis_timeseries=[
        # Take a few thermal and nonthermal and bkg energy bands and sum them
        # together to make the basis timeseries.
        # This helps with statistics and also makes it so that
        # no single energy band dominates the behavior of the emission
        sliced["cts"][th_idx - 1 : th_idx + 2].sum(axis=0),
        sliced["cts"][nth_idx - 1 : nth_idx + 2].sum(axis=0),
        sliced["cts"][bkg_idx - 1 : bkg_idx + 2].sum(axis=0),
    ],
    # In this case, because we are using a background light curve,
    # we want no constant offset.
    constant_offset=False,
)

# Decompose the data using the `decomp` module
# See the docstring for more info
decomposed = decomp.bootstrap(
    dp=pack, errors=sliced["cts_err"], num_iter=3000, clip_negative=True
)

## Plot the decomposed data on top of the original data

In [ ]:
# As per the docstring, the decomposed data are the first N-1 entries, while the
# intercept is included in the final entry.
index_map = {"thermal": 0, "nonthermal": 1, "background": 2}

# Energy binning is nonuniform, so we divide it out
# for a nicer looking spectrum
de = np.diff(sliced["energy_bins"]) << u.keV

# Average the live time across all detector pairs
dt = np.sum(np.diff(sliced["time_bins"])).to(u.s)

fig, ax = plt.subplots(layout="constrained")

all_counts = sliced["cts"].sum(axis=1)
all_errors = np.sqrt(np.sum(sliced["cts_err"] ** 2, axis=1))

with quantity_support():
    stairs_with_error(
        sliced["energy_bins"],
        all_counts / de / dt,
        all_errors / de / dt,
        ax=ax,
        label="data",
        line_kw={"color": "black"},
    )
    for label, index in index_map.items():
        # Get the samples associated with your pseudobasis
        samples = decomposed[:, index, :]
        avg_cts = np.mean(samples, axis=0) << u.ct
        avg_std = np.std(samples, axis=0) << u.ct

        stairs_with_error(
            sliced["energy_bins"],
            avg_cts / de / dt,
            avg_std / de / dt,
            ax=ax,
            label=label,
        )

ax.legend()
ax.set(
    title="RHESSI decomposition",
    xscale="log",
    yscale="log",
    xlim=(4, 300),
    ylim=(0.01, 2e3),
)

### We can see that the decomposed data captures the various components well.
### This is equivalent to Figure 4 from the paper.